# Experiment 2: Code Solutions

## Where we're at
Up to this point we've coded up some basic infrastructure that makes it easy to run ARC prize items, where out business logic including Reflexion and giving the agent plenty of time to think fits into about a page.

We've run the first 20 problems in the training set and got 3/20 = 15%.  This is showing some non-zero signs of life and is a good baseline to start improving.

## Where we go from here
Looking at the sessions where it failed, it looks like it understands what to do but is having trouble with the details of the execution.  For example, on one of the items involving a flood fill, the agent understands what it needs to do and mostly does it, but has trouble geting every single cell right.

## Hypothesis: Python DSL
Can we do a thing where it writes code to do it?  Others on the leaderboard have done this using their own DSL and fine tuning.  Maybe we can make a collection of Python functions with libraries that make a lot of these tasks as easy as practical.  This way we leverage the model's deep understanding of Python.


## Next steps:
* [ ] **Verify agent understanding**: Does it actually understand the patterns? Review the cases where it failed.  If it can code strong, will that actually help? It won't help if the agent did not actually understand the pattern.  My hypothesis is grounded in the assumption that mostly it does understand what it needs to do, but it is struggling with the **execution** of what it needs to do.  Verify this assumption.
* [ ] **Prompt**: Put together a prompt including what you propose doing, and see how the LLM does with that.  This way I can verify the promise of this approach before I actually build all of it. This is a common trick I use with LLM projects: verify the promise of the approach before I build the infrastructure needed to make it happen.
* [ ] **Evaluation Infrastructure:** I should set up an infrastructure where I can rapidly try different approaches, leaving the old approaches intact maybe, and rapidly get feedback about how that worked out.  For example, retry the same set of 20 problems.  At the moment scores are stored in each session, but I should also create a summary score sheet stored as both json as well as markdown, showing the summary of what happened in the run.
    * **Score sheet** for a run so we can quickly see which ones worked and didn't.  We want this to come out as a pandas dataframe so that I can rapidly filter and sort and such.  I may actually add human annotations, e.g. "flood fill" so I know what the strategy on the case is at a glance.  Because it's a pandas dataframe I'll probably save these as a csv.  Make a function that returns a markdown representation of the results.  Join in the descriptions of what the case is.  Make another function that makes all that for the last run that we did, so I can easily plop that into a notebook.  Could include [links to the cases like this](https://arcprize.org/play?task=06df4c85).
    * **Images to improve human understanding**.  If I struggle to understand a given case by staring at the matrix then it may be worth the time investment to make img files that show the input and output cases to help me iterate on this faster.  I say *human* understanding because it's not clear to me that image recognition for the multi-modal models actually improves performance much.  Evidence says no, but I'm not totally sure.  If our agent understands the patterns of most of our failed use cases then that will be evidence that the image recognition doesn't matter too much to the agent.  I have observed that the LLM has no trouble at all understanding a big jumbled mess of unformatted JSON in a way that's superhuman, so it sure wouldn't surprise me if the image recognition doesn't help much.  So this is why I say the images would be to aid me in *human* understanding, rapidly remembering what the case is about when looking at a case.
    * [ ] **Image experiment**.  In some cases, like #9 below, it seems like the agent didn't grok what's happening.  Would image recognition help in such cases? Run an experimental prompt with an image and see if it does a better job getting it with that prompt.
* [ ] **Coding Infrastructure:**  Assuming the above two things turn out like I expect, some ideas of what to do with it are:
    * **Disjoint sets**.  Make a great abstraction for working with disjoint sets, as I have done in the past for working with datacenter tiles.
    * **Coding agent.**  Make a coding agent
    * **Module describer.** Make a thing that describes all the public classes and methods of a module, so that I'm spending more time writing code and less time writng prompts.

## Verify Agent Understanding

In this section we'll review the cases and see if it looks like the LLM understood what the case was about. (I should be able to just push a button and get the dataframe or markdown of the last run; maybe I'll implement that later.)

| Seq | Case | Result | What is it | Did agent get it | Remarks |
|-----|------|--------|------------|-------------------|---|
| 1 | [007bbfb7](https://arcprize.org/play?task=007bbfb7) |fail | 3x3 expand to 9x9, replicate when set | No | [1] |
|2  | [00d62c1b](https://arcprize.org/play?task=00d62c1b) |fail | Flood fill | |
|3  | [017c7c7b](https://arcprize.org/play?task=017c7c7b) | PASS | Continue pattern to height 9 | Yes |
|4  | [025d127b](https://arcprize.org/play?task=025d127b) | PASS | Square up the bottom | Yes | [4] |
|5  | [045e512c](https://arcprize.org/play?task=045e512c) | fail | Directional color replicator | No? |  [5] |
|6  | [0520fde7](https://arcprize.org/play?task=0520fde7) | fail | Boolean AND on two 3x3s | |
|7  | [05269061](https://arcprize.org/play?task=05269061) | fail | Diagonal pattern expander | No~ | [7] |
|8  | [05f2a901](https://arcprize.org/play?task=05f2a901) | fail | Square sucks in the shape | |
|9  | [06df4c85](https://arcprize.org/play?task=06df4c85) | fail | Connect same colors in the grid | No.|  Thought about symmetry erroneously. Would image help? |
|10 | [08ed6ac7](https://arcprize.org/play?task=08ed6ac7) | fail | Size 1-4 histogram classifier | No | [10] |
|11 | [09629e4f](https://arcprize.org/play?task=09629e4f) | fail | Find, copy and expand the section without blue | |
|12 | [0962bcdd](https://arcprize.org/play?task=0962bcdd) | fail | Grow crystals | |
|13 | [0a938d79](https://arcprize.org/play?task=0a938d79) | fail | Expand into stripes and continue | No | Didn't realize it could go either rows or columns.[13] |
|14 | [0b148d64](https://arcprize.org/play?task=0b148d64) | fail | Find the unique color cluster and output it | | [14] |
|15 | [0ca9ddb6](https://arcprize.org/play?task=0ca9ddb6) | fail | Grow crystals 2, don't grow blue | |
|16 | [0d3d703e](https://arcprize.org/play?task=0d3d703e) | fail | Simple color transform | No | [16] |
|17 | [0dfd9992](https://arcprize.org/play?task=0dfd9992) | PASS | Ginormous pattern fill | Yes |
|18 | [0e206a2e](https://arcprize.org/play?task=0e206a2e) | fail | Rotate, translate ships to indicated positions | No | [18] |
|19 | [10fcaaa3](https://arcprize.org/play?task=10fcaaa3) | fail | Grow blue on diagonals and double the pattern | |
|20 | [11852cab](https://arcprize.org/play?task=11852cab) | fail | Expand to a square with given color |  No | [20] |

[1] It understood the idea that the 3x3 pattern is replicated, but missed the idea that we replicate it only to destination cells where the corresponding cell is not zero in the input.  Maybe this would be better with code.  But also Hypothesis 16 below could apply, because it didn't seem to have a crystal clear algorithm.

[4] I didn't 100% grok its reasoning, but A) I struggled hard with understanding this one when I did it personally, and B) it's hard to argue with success.

[5] It did understand the pattern from the 2 cases I gave it. But note that on this case the first two test examples go vertical and horizontal, while the last of the 2 use cases goes diagnonally, which is considerably more complex. This messed it up and it didn't know how to deal with it. Hypothesis: it might have done better if I didn't have a holdout case, because the first tests do not generalize to the third test. Ooops.  Might have done better if I just gave it all 3 at once.  I'm unsure whether to say it generalized or not. Seems like maybe it did.
    * Checking this hypothesis. Trying manually crafted prompts that do no holdout and give it all training examples at once.  It didn't grok the pattern enough to pull this off.  Maybe it understood it well enough to do it if it wrote code? Unsure.

[7] It got that the pattern repeats, but failed to grasp that the starting point for the pattern is expanding upon the existing given pattern.  It takes some thought to figure out how to make the pattern apply in a way that the output overlays with the input, and it never got that.

[10] It did not understand that the color transform is based on relative sizes of the bars.  It didn't seem to recognize that this is like a bar chart or histogram.

[13] I repeated the same prompts but attached an image. It then seemed to understand that sometimes the pattern is in rows and sometimes the pattern is in columns.  It still didn't get it right.

[14] It realized that it needs to replicate a submatrix, but never achieved clarity on how to select which submatrix to extract.  Perhaps this would be helped by the prompt idea in hypothesis [16].

[16] This one is a simple color transform; a given color on the input *always* transforms to something on the output.
    * [ ] Hypothesis: it never did have a solid theory on how to do this that applies to all input cases.  Could try an additional step that is like
    > "Do you have a solid, unambiguous theory of operation or algorithm that applies to **all** of the cases?  If your hypothesis is not strong, rethink your approach and come up with something that cleanly explains all of the use cases you know about.  Experience has taught us that one of the most common ways we end up getting problems wrong is by trying to do these problems with an ambiguous algorithm.  Without crystal clarity on the rule we will almost certainly fail.  So this is your chance to think about it if your algorithm does not have sufficient clarity."
    This hypothesis might also help with these other cases where it didn't seem to have a clear idea of what it was doing: 5, 10, 18, 1, 16, 14
    * [ ] Once you get an experimentation framework going try this out.
    * [ ] Could be that we need to have it tell us if it is able to achieve clarity and is ready to try the next step or not.  Then we can go for some real deepr thinking.

[18] It never fully grasped the central idea that some of these points indicate the position and rotation of the newly transformed thing. Maybe vision would help? No, vision did not help.

[20] It never grokked that it needs to expand into a square. "The correct logic was to place certain elements according to more intricate patterns," it said.

* [ ] Should we try using o1?

#### Success: `0dfd9992` Ginormous pattern fill
Can we just take a moment to bask in the glory that it actually solved #17 above? That's this one:
![ginormous pattern fill](img/big-pattern-continue.png)

So this thing may not be storming up the leaderboards, but it definitely has some intelligence.  Considering how 15% seems so terrible, it's basically hard for me to believe that it got this one right. So I'm going to go look at its transcript to verify I didn't make some kind of clerical error in thinking it did this one successfully:


> Strategy for Future Problems

To maintain accuracy and efficiency when tackling similar problems in the future, here are some useful strategies and insights:

1. **Pattern Recognition:**
   - Always identify and firmly understand the repeating patterns in the input matrix. It’s crucial to map out these periodic structures before attempting any transformation.
   
2. **Zero Replacement Strategy:**
   - When filling zeros, ensure you access the most localized pattern surrounding them to seamlessly continue the observed repetition.

3. **Thorough Testing:**
   - Apply the method across several hypothetical placements or examples to validate that the identified approach is uniformly applicable.

4. **Documentation:**
   - Note detailed observations on paper or a digital pad during the problem-solving process, especially identifying edges where patterns restart or loop.

5. **Iterative Validation:**
   - Consider each row and column, checking if the placement maintains cohesion in the pattern extended beyond immediate neighboring cells.

By ensuring these steps, any overlooked or subtle nuances in the pattern can be spotted, and solutions replicated more effectively across other similar problem sets.


> User
Let's try another problem, this one is from the test set, the real exam.
Here is the input pattern:

> Test Case Input 0: size 21x21
```
9 6 5 6 9 5 3 3 5 9 6 5 6 9 5 3 3 5 9 6 5
6 3 2 3 6 2 9 9 2 0 0 0 0 0 2 9 9 2 6 3 2
5 2 1 2 5 1 8 8 1 0 0 0 0 0 1 8 8 1 5 2 1
6 3 2 3 6 2 9 9 2 0 0 0 0 0 2 9 9 2 6 3 2
9 6 5 6 9 5 3 3 5 9 6 5 6 9 5 3 3 5 9 6 5
5 2 1 2 5 1 8 8 1 5 2 1 2 5 1 8 8 1 5 2 1
3 9 8 9 3 8 6 6 8 3 9 8 9 3 8 6 6 8 3 9 8
3 9 8 9 3 8 6 6 8 3 9 8 9 3 8 6 6 8 3 9 8
5 2 1 2 0 0 0 0 1 5 2 1 2 5 1 8 8 1 5 2 1
9 6 5 6 0 0 0 0 5 9 6 5 6 9 5 3 3 5 9 6 5
6 3 2 3 0 0 0 0 2 6 3 2 3 6 2 9 9 2 6 3 2
5 2 1 2 5 1 8 8 1 5 2 1 2 5 1 8 8 1 5 2 1
6 3 2 3 6 2 9 9 2 6 3 2 3 6 0 0 9 2 6 3 2
9 6 5 6 9 5 3 3 5 9 6 5 6 9 0 0 3 5 9 6 5
5 2 1 2 5 1 8 8 1 5 2 1 2 5 0 0 8 1 5 2 1
0 0 8 9 0 0 0 6 8 3 9 8 9 3 0 0 6 8 3 9 8
0 0 8 9 0 0 0 6 8 3 9 8 9 3 8 6 6 8 3 9 8
5 2 1 2 0 0 0 8 1 5 2 1 2 5 1 8 8 1 5 2 1
9 6 5 6 0 0 0 3 5 9 6 5 6 9 5 3 3 5 9 6 5
6 3 2 3 0 0 0 9 2 6 3 2 3 6 2 9 9 2 6 3 2
5 2 1 2 5 1 8 8 1 5 2 1 2 5 1 8 8 1 5 2 1
```

Please output a solution for this one.



> Assistant
*Structured result*:
`rows=[[9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5], [6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2], [9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8], [3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5], [6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2], [9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8], [3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8, 9, 3, 8, 6, 6, 8, 3, 9, 8], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1], [9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5, 6, 9, 5, 3, 3, 5, 9, 6, 5], [6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2, 3, 6, 2, 9, 9, 2, 6, 3, 2], [5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1, 2, 5, 1, 8, 8, 1, 5, 2, 1]]`


> System

> Agent Answer: `PASS`